In [1]:
!uv pip install --system polars 
!uv pip install --system polars_ds 
!uv pip install --system xgboost 
!uv pip install --system lightgbm 
!uv pip install --system plotnine 
!uv pip install --system duckdb

Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 10ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 10ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 10ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 10ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 13ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 9ms


In [2]:
import polars as pl 
import duckdb
import polars_ds as pds
from datetime import datetime
import xgboost as xgb
import lightgbm as lgb
import itertools
import gc

from plotnine import (
    ggplot,
    aes,
    geom_line,
    facet_wrap,
    labs,
    scale_x_datetime,
    element_text,
    theme_538
)

## Read the data

In [3]:
# Training data, scan the data
df_train = pl.read_csv('sales_train.csv')
f'Rows: {df_train.select(pl.len()).item()}, Columns: {len(df_train.columns)}'

'Rows: 4007419, Columns: 14'

In [4]:
# Cast to date
df_train = df_train.with_columns(pl.col('date').cast(pl.Date))

# Inspect the data
df_train.head()

unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4845,2024-03-10,"""Budapest_1""",6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,0.15312,0.0,0.0
4845,2021-05-25,"""Budapest_1""",4663.0,12.63,455.96,1.0,0.0,0.0,0.0,0.0,0.15025,0.0,0.0
4845,2021-12-20,"""Budapest_1""",6507.0,34.55,455.96,1.0,0.0,0.0,0.0,0.0,0.15025,0.0,0.0
4845,2023-04-29,"""Budapest_1""",5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4845,2022-04-01,"""Budapest_1""",5997.0,35.92,486.41,1.0,0.0,0.0,0.0,0.0,0.15649,0.0,0.0


In [5]:
# Test data
df_test = pl.read_csv('sales_test.csv')
f'Rows: {df_test.select(pl.len()).item()}, Columns: {len(df_test.columns)}'

# Cast to date
df_test = df_test.with_columns(pl.col('date').cast(pl.Date))

# Inspect the data
df_test.head()

df_test.sort(by = ['unique_id','date'])

unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2024-06-03,"""Frankfurt_1""",1725.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-06-04,"""Frankfurt_1""",1566.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-06-05,"""Frankfurt_1""",1643.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-06-06,"""Frankfurt_1""",1763.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-06-07,"""Frankfurt_1""",2023.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…
5431,2024-06-12,"""Prague_2""",6108.0,81.61,0.39911,0.0,0.0,0.0,0.0,0.0,0.0
5431,2024-06-13,"""Prague_2""",6679.0,81.61,0.39911,0.0,0.0,0.0,0.0,0.0,0.0
5431,2024-06-14,"""Prague_2""",6696.0,136.47,0.39911,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
inventory = pl.read_csv('inventory.csv')

inventory.head()

unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
i64,i64,str,str,str,str,str,str
5255,2583,"""Pastry_196""","""Bakery""","""Bakery_L2_14""","""Bakery_L3_26""","""Bakery_L4_1""","""Prague_3"""
4948,2426,"""Herb_19""","""Fruit and vegetable""","""Fruit and vegetable_L2_30""","""Fruit and vegetable_L3_86""","""Fruit and vegetable_L4_1""","""Prague_3"""
2146,1079,"""Beet_2""","""Fruit and vegetable""","""Fruit and vegetable_L2_3""","""Fruit and vegetable_L3_65""","""Fruit and vegetable_L4_34""","""Prague_1"""
501,260,"""Chicken_13""","""Meat and fish""","""Meat and fish_L2_13""","""Meat and fish_L3_27""","""Meat and fish_L4_5""","""Prague_1"""
4461,2197,"""Chicory_1""","""Fruit and vegetable""","""Fruit and vegetable_L2_17""","""Fruit and vegetable_L3_33""","""Fruit and vegetable_L4_1""","""Frankfurt_1"""


In [7]:
# Same product id, but different unique_id across warehouse
inventory.filter(pl.col('product_unique_id')==1309)

unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
i64,i64,str,str,str,str,str,str
2623,1309,"""Donut_23""","""Bakery""","""Bakery_L2_14""","""Bakery_L3_76""","""Bakery_L4_1""","""Prague_1"""
2622,1309,"""Donut_23""","""Bakery""","""Bakery_L2_14""","""Bakery_L3_76""","""Bakery_L4_1""","""Prague_3"""
2624,1309,"""Donut_23""","""Bakery""","""Bakery_L2_14""","""Bakery_L3_76""","""Bakery_L4_1""","""Prague_2"""
2621,1309,"""Donut_23""","""Bakery""","""Bakery_L2_14""","""Bakery_L3_76""","""Bakery_L4_1""","""Brno_1"""


In [8]:
# Read the calender dataset
calendar = pl.read_csv('calendar.csv')

# Cast to date
calendar = calendar.with_columns(pl.col('date').cast(pl.Date))
calendar.head()


date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
date,str,i64,i64,i64,i64,str
2022-03-16,null,0,0,0,0,"""Frankfurt_1"""
2020-03-22,null,0,0,0,0,"""Frankfurt_1"""
2018-02-07,null,0,0,0,0,"""Frankfurt_1"""
2018-08-10,null,0,0,0,0,"""Frankfurt_1"""
2017-10-26,null,0,0,0,0,"""Prague_2"""


In order to join this to the training dataset one needs to take into account not only the date, but also warehouse

In [9]:
# This date have duplicates. 
calendar.filter(pl.col('date')==datetime(2022,3,16)) 

date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
date,str,i64,i64,i64,i64,str
2022-03-16,null,0,0,0,0,"""Frankfurt_1"""
2022-03-16,null,0,0,0,0,"""Budapest_1"""
2022-03-16,null,0,0,1,0,"""Prague_1"""
2022-03-16,null,0,0,0,0,"""Munich_1"""
2022-03-16,null,0,0,0,0,"""Prague_3"""
2022-03-16,null,0,0,1,0,"""Prague_2"""
2022-03-16,null,0,0,0,0,"""Brno_1"""


Join tables

In [10]:
# Join calendar days
df_train = (df_train

# Join calendar days
.join(other = calendar.drop('holiday_name'),
      on = ['date','warehouse'],
      how = 'left')

# Join inventory
.join(other = inventory.drop('warehouse'),
      on = ['unique_id'],
      how = 'left'
     )
)


## Inspect the training data

In [11]:
# There are 5390 unique_id (time series)
df_train.select(pl.col('unique_id').n_unique())

unique_id
u32
5390


In [12]:
# Check the time series length for each unique id and warehouse
(df_train
.group_by('unique_id')
.agg(

     # Time series length
     pl.len().alias('n'),

     # Calculate min and max year
     pl.col('date').dt.year().max().alias('min_year'),
     pl.col('date').dt.year().max().alias('max_year')
    )
.head()
)

unique_id,n,min_year,max_year
i64,u32,i32,i32
2763,189,2024,2024
4999,32,2024,2024
2891,808,2024,2024
5121,543,2023,2023
2641,67,2023,2023


According to the table, every time series have different lengths. Some time series have only data for 2023, while others have for both 2023 and 2024.

In [13]:
# Check if there are any gaps in the time series
(df_train
 .sort(by = ['warehouse','unique_id','date'],descending = [False,False,False])
 .select('unique_id','date','warehouse')

 # Obtain the previous day
 .with_columns(pl.col('date').shift(1).over('warehouse','unique_id').alias('date_lag'))

 .filter(pl.col('date_lag').is_not_null())
 
 # Calculate the difference
 .with_columns(pl.col('date')
               .sub(pl.col('date_lag'))
               .dt.total_days()
               .alias('diff')
              )
 # Filter on gaps
 .filter(pl.col('diff')>1)
 .select(pl.col('unique_id').n_unique())

 
)


unique_id
u32
5238


Almost all unique_id have gaps the time series

In [14]:
# Null count
(df_train
.null_count()
)

unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount,holiday,shops_closed,winter_school_holidays,school_holidays,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# Check the null values 
(
df_train
.sort(by = ['warehouse','unique_id','date'])
.filter(pl.col('unique_id')==612)
.select('holiday','shops_closed','winter_school_holidays','school_holidays')
.sum()
)

holiday,shops_closed,winter_school_holidays,school_holidays
i64,i64,i64,i64
13,10,6,23


## Model setup

In [16]:
# Sort the data
df_train = df_train.sort(by = ['warehouse','unique_id','date'],descending = [False,False,False])


Create features

In [17]:
#Multiple discounts with different type can be run at 
#the same time, but always the highest possible
#discount among these types is used for sales. 
#Negative discount value should be interpreted as no discount. 
df_train = (df_train
.with_columns(pl.max_horizontal(*[c for c in df_train.columns if 'type' in c]).alias('max_discount'))

)


In [18]:

# Cumulative mean/expanding mean per unique_id
(df_train
.select('unique_id',pl.lit(1).alias('cum_count'),'sales')
.with_columns(pl.col('cum_count')
              .cum_sum()
              .over('pri'),
              
              pl.col('sales')
              .cum_sum()
              .over('unique_id')
              .alias('cum_sum_sales'))
 
.with_columns(pl.col('cum_sum_sales')
              .truediv(pl.col('cum_count'))
              .alias('expanding_mean')
             )
.with_columns(pl.col('expanding_mean')
              .shift(1)
              .over('unique_id')
              .alias('expanding_mean')
)
)


#(34.09 + 45.14+13+55.62)/3

ColumnNotFoundError: pri

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["unique_id", "cum_count", "sales"]; PROJECT */3 COLUMNS; SELECTION: None

In [19]:
# Training proportion
training_proportion = 0.75

# Number of lags
lag_n   = 2

# Create lag expressions
lag_exp = [pl.col('sales').shift(n = lag_n).over('unique_id','warehouse').alias(f'sales_lag_{lag_n}') for lag_n in range(1,lag_n+1)]

# lag columns names used for filter on non null values
lag_exp_columns_names = [f'sales_lag_{lag_n}' for lag_n in range(1,lag_n+1)]

# Define ml dataset
ml_data = (df_train

.drop_nulls(subset = 'sales')
# Extract week, day and month
.with_columns(pl.col('date').dt.week().cast(pl.Int64).alias('week'),
              pl.col('date').dt.day().cast(pl.Int64).alias('day'),
              pl.col('date').dt.month().cast(pl.Int64).alias('month'))
           
# Training proportion per symbol id
.with_columns(pl.lit(training_proportion).alias('training_proportion'))

# Time series length
.with_columns(pl.len().over('unique_id').alias('n'))

# Creating observations id
.with_columns(pl.lit(1).alias('obs_id'))
.with_columns(pl.col('obs_id').cum_sum().over('unique_id'))

# Calculate the 75th obs_id.
.with_columns(pl.col('training_proportion')
        .mul(pl.col('n'))
        .cast(pl.Int64)
        .alias('training_size'))

           
)


# Filter training data and create lags
X_train = (ml_data

# Create lags
.with_columns(*lag_exp)
                    
.filter(pl.col('obs_id')<=pl.col('training_size'))
                    
# Only containing complete values on lag variables
.drop_nulls(subset = lag_exp_columns_names)


)

# Relevant features
features_train = ['date','unique_id','sales',
                  'week','day','month','sell_price_main',
                  'total_orders','max_discount',*lag_exp_columns_names]

# Calendar features
features_calendar = calendar.drop('holiday_name','date','warehouse').columns

# Use this list to sort the columns names in the prediction step. 
features_ordering = ['week','day','month','sell_price_main',
                     'total_orders','max_discount',
                     *lag_exp_columns_names] + features_calendar

# Define X_train and y_train
y_train = X_train.select('sales')
X_train = X_train.select(*features_train,*features_calendar).drop('sales')

# Filter test data
X_test = (ml_data
               .filter(pl.col('obs_id')>pl.col('training_size'))
               
               # Create new row id to remove null values on lag
               .with_columns(pl.lit(1).alias('obs_id'))
               .with_columns(pl.col('obs_id').cum_sum().over('unique_id'))
               
                
               )
# Lags needs to be joined sequentially from prediction
features_test = ['date','unique_id','sales',
                 'week','day','month','sell_price_main',
                 'total_orders','max_discount']

# Define test data
y_test = X_test.select('sales')
X_test = X_test.select(*features_test,*features_calendar).drop('sales')


In [20]:
# Initialize model
model = lgb.LGBMRegressor(n_estimators=100, 
                         learning_rate=0.1,
                         random_state = 123,
                         objective='regression',
                         metric='mae')



# Train the model
model.fit(
    X_train.select(*features_ordering), y_train
    #eval_set=[(X_val, y_val)],  # Evalueringssett
    #eval_metric="rmse",        # Evalueringsmetrikk
    #early_stopping_rounds=10,  # Stopp hvis ytelsen ikke forbedres etter 10 iterasjoner
    #verbose=True               # Skriv ut metrikker under trening
)

c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1382
[LightGBM] [Info] Number of data points in the train set: 2992724, number of used features: 12
[LightGBM] [Info] Start training from score 106.708747


LGBMRegressor(metric='mae', objective='regression', random_state=123)

In [21]:
# Obtain predicted value on training data
y_pred_training = pl.from_numpy(model.predict(X_train.select(*features_ordering))).select(pl.all().alias('y_pred'))
y_pred_training  = pl.concat([y_pred_training,X_train.drop('sales_lag_1','sales_lag_2')],how = 'horizontal')

# Create lag expressions
lag_exp = [pl.col('y_pred').shift(n = lag_n).over('unique_id').alias(f'y_pred_{lag_n}') for lag_n in range(1,lag_n+1)]

# lag columns names used for filter on non null values
lag_exp_columns_names = [f'y_pred_{lag_n}' for lag_n in range(1,lag_n+1)]

# Create lagged features based on predicted values, 
# and retain the two last observations from each unique_id series.
# The number of observations to retain are depending on the
# number of lag values. Also join the real sales values.
y_pred_training = (y_pred_training
                  .sort(by = ['unique_id','date'])
                  .with_columns(*lag_exp)
                  .group_by('unique_id')
                  .tail(lag_n)
                  .join(other = (df_train
                                .select('unique_id','date','sales')
                                .rename({'sales':'y_true'})
                                ),
                        on = ['unique_id','date'],
                        how = 'left'
                       )
                  )

y_pred_training.filter(pl.col('unique_id')==123)


unique_id,y_pred,date,week,day,month,sell_price_main,total_orders,max_discount,holiday,shops_closed,winter_school_holidays,school_holidays,y_pred_1,y_pred_2,y_true
i64,f64,date,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64
123,247.294197,2024-01-21,3,21,1,2198.84,6384.0,0.1298,0,0,0,0,217.451251,146.892681,114.91
123,147.512593,2024-01-22,4,22,1,2198.84,6378.0,0.0,0,0,0,0,247.294197,217.451251,157.62


Initialize dataset for forecast engine. Forecasting the first value from each series in the test set.

In [22]:
forecast_initializer = (X_test

# Filter on the first value per series
.group_by('unique_id')
.head(1)

# Join the two last values from training data
.join(other = y_pred_training.select('unique_id','y_true','date'),
      on = ['unique_id'],
      suffix = "_pred"
     )
#.filter(pl.col('unique_id')==123)

# Sort the dataset
.sort(by = ['unique_id','date','date_pred'],descending = [False,False,False])

 # Create index values for lag
.with_columns(pl.lit(1).alias('lag'))
.with_columns(pl.col('lag').cum_sum().over('unique_id','date'))

# Replace with string values
.with_columns(pl.col('lag')
              .cast(pl.String)
              .replace(1,"lag_2")
              .replace(2,"lag_1")
             )
# Pivot
.pivot(index = [*[c for c in features_test if c != 'sales'],
               *features_calendar],
       on = 'lag',
      values = 'y_true')

# Cast to f64 because the training uses this type
.with_columns(pl.col(['lag_1','lag_2']).cast(pl.Float64))
.rename({'lag_1':'sales_lag_1','lag_2':'sales_lag_2'})

)

forecast_initializer.filter(pl.col('unique_id')==123).head()




date,unique_id,week,day,month,sell_price_main,total_orders,max_discount,holiday,shops_closed,winter_school_holidays,school_holidays,sales_lag_2,sales_lag_1
date,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,f64
2024-01-23,123,4,23,1,2198.85,6498.0,0.0,0,0,0,0,114.91,157.62


Forecast the first values

In [23]:
# Predictions 
forecast_initializer_predictions = pl.from_numpy(model.predict(forecast_initializer.select(*features_ordering)))

# Rename columns
forecast_initializer_predictions = forecast_initializer_predictions.rename({'column_0':'y_pred'})

# Hstack
forecast_initializer = pl.concat([forecast_initializer,forecast_initializer_predictions],
                                how = 'horizontal')

# Get y_true
forecast_initializer = forecast_initializer.join(df_train.select('unique_id','date',pl.col('sales').alias('y_true')),
                                                on = ['unique_id','date'],
                                                how = 'left')


forecast_initializer.filter(pl.col('unique_id')==123)

date,unique_id,week,day,month,sell_price_main,total_orders,max_discount,holiday,shops_closed,winter_school_holidays,school_holidays,sales_lag_2,sales_lag_1,y_pred,y_true
date,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64
2024-01-23,123,4,23,1,2198.85,6498.0,0.0,0,0,0,0,114.91,157.62,143.760615,168.7


Now loop through the rest of the predictions sequentially

In [ ]:
# Join the lag_
test_123 = (X_test
.join(other = (forecast_initializer
      .select('date','unique_id','y_pred','sales_lag_1','sales_lag_2')
              ),
      on = ['date','unique_id'],
      how = 'left'
     )
.sort(by = ['unique_id','date'])
#.filter(pl.col('unique_id')==0)

# using row id to select specific rows
.with_row_index(name = 'row_id',offset = 0)
)


def predict_func()

1003863

In [32]:
test_123.filter(pl.col('unique_id')==1).head(1)

row_id,date,unique_id,week,day,month,sell_price_main,total_orders,max_discount,holiday,shops_closed,winter_school_holidays,school_holidays,y_pred,sales_lag_1,sales_lag_2
u32,date,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64
9,2024-04-14,1,15,14,4,2.87,1204.0,0.0,0,0,0,1,80.235913,77.68,104.57


In [241]:
# Create a forecast engine function. 
def forecast_engine(df: pl.DataFrame):
    
    # using row id to select specific rows
    df = df.with_row_index(name = 'row_id',offset = 0)
    
    # Loop over all observations 
    for index in range(0, df.shape[0]):

        # Loop over the first observation (it is already predicted) 
        if df.select(pl.col('date'))[index].item() != df.select(pl.col('date').min()).item():
            
            # Obtain lag 1 
            sales_lag_1 = df[index-1].select('y_pred').item()

            # Obtain lag 2
            sales_lag_2 = df[index-1].select('sales_lag_1').item()

            # Update dataset with lag values
            df = (df
                  .with_columns(
                      # Lag 1 
                      pl.when(pl.col('row_id') == index).then(pl.lit(sales_lag_1)).otherwise(pl.col('sales_lag_1')).alias('sales_lag_1'),
                      
                      # Lag 2
                      pl.when(pl.col('row_id') == index).then(pl.lit(sales_lag_2)).otherwise(pl.col('sales_lag_2')).alias('sales_lag_2')
                  )
                  )

            # Make prediction
            y_pred = model.predict(df.select(*features_ordering))

            # Put the prediction into the dataset
            df = df.with_columns(pl.when(pl.col('row_id')==index).then(pl.lit(y_pred)).otherwise(pl.col('y_pred')).alias('y_pred'))
         
    df = df.drop('row_id')
    return(df)

Initialize lags on the first observation in the test set

In [242]:
forecast_engine(test[0])



date,unique_id,week,day,month,sell_price_main,total_orders,max_discount,holiday,shops_closed,winter_school_holidays,school_holidays,y_pred,sales_lag_1,sales_lag_2
date,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64
2023-07-20,0,29,20,7,853.2,5673.0,0.30025,0,0,0,0,329.700272,359.02,85.95
2023-07-21,0,29,21,7,853.2,5700.0,0.30025,0,0,0,0,355.54896,329.700272,359.02
2023-07-22,0,29,22,7,853.2,5209.0,0.30025,0,0,0,0,364.786646,355.54896,329.700272
2023-07-23,0,29,23,7,853.2,5262.0,0.30025,0,0,0,0,373.738864,364.786646,355.54896
2023-07-24,0,30,24,7,853.2,5661.0,0.30025,0,0,0,0,378.127859,373.738864,364.786646
2023-07-25,0,30,25,7,853.2,5637.0,0.30025,0,0,0,0,390.88081,378.127859,373.738864
2023-07-26,0,30,26,7,853.2,5485.0,0.0,0,0,0,0,366.701766,390.88081,378.127859
2023-07-27,0,30,27,7,853.2,5490.0,0.0,0,0,0,0,353.948816,366.701766,390.88081
2023-07-28,0,30,28,7,853.2,5621.0,0.0,0,0,0,0,339.380885,353.948816,366.701766


In [ ]:
for index in range(0,len(test)):
     forecast_engine(test[index])
     print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27